# Aviation Exploration [placeholder title]

## Goals

PLACE HOLDER TO ADD GOALS OF THE PROJECT

In [1]:
# All imports
# reminder to add ignore warning code at the end

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Data

In [2]:
# Open data file

# Had to specify encoding 'latin-1' for file to load
# Setting low_memory=False in the read_csv function will force pandas to read the entire file at once, 
# which can help with inferring the correct data types. 

ad = pd.read_csv('Data/AviationData.csv', encoding='latin-1', low_memory=False)

#### Initial Exploration

In [3]:
# Get initial info

ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [4]:
ad.sample(n=10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
71548,20120331X34152,Accident,ANC12FA028,2012-03-31,"Sleetmute, AK",United States,615627N,1573713W,NaN,NaN,...,Personal,RICHARD I. WILMARTH,2.0,0.0,0.0,0.0,VMC,NaN,Operation of the airplane by a non-certificate...,25-09-2020
17773,20001213X31341,Accident,SEA87LA117,1987-06-14,"RENO, NV",United States,NaN,NaN,NaN,AIR SAILING,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,Landing,Probable Cause,05-07-1988
78332,20160603X45936,Accident,ERA16LA202,2016-06-03,"Mountain City, TN",United States,362829N,0814817W,NaN,NaN,...,NaN,PHI AIR MEDICAL,0.0,1.0,0.0,3.0,VMC,NaN,An incorrect and unapproved repair of the engi...,25-09-2020
31940,20001211X16171,Accident,LAX93FA066,1992-12-14,"OCEANSIDE, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,VMC,Cruise,Probable Cause,26-08-1993
68985,20100924X30155,Accident,CEN10FA557,2010-09-24,"Woodruff, WI",United States,046129N,0894420W,ARV,Lakeland Airport,...,Personal,"JA Aero, Inc.",2.0,0.0,0.0,0.0,IMC,NaN,The pilot's decision to attempt an instrument ...,25-09-2020
52455,20020429X00587,Accident,MIA02LA087,2002-04-21,"EHRHARDT, SC",United States,33.005556,-81.048333,SC55,Broxton Bridge Plantation,...,Personal,NaN,NaN,NaN,NaN,3.0,VMC,Landing,Probable Cause,26-08-2002
9474,20001214X40910,Accident,SEA84LA208,1984-08-21,"SHELLEY, ID",United States,NaN,NaN,NaN,NaN,...,Aerial Application,NaN,0.0,1.0,0.0,0.0,VMC,Cruise,Probable Cause,NaN
78131,20160424X92047,Accident,ERA16FA168,2016-04-24,"Belton, SC",United States,343458N,0082219W,SC82,OAKHILL AIRPARK,...,Personal,NaN,1.0,0.0,0.0,0.0,VMC,NaN,A loss of engine power for reasons that could ...,25-09-2020
36297,20001206X02452,Accident,FTW95LA032,1994-10-27,"RUSTON, LA",United States,NaN,NaN,5L3,RUSTON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,Maneuvering,Probable Cause,25-04-1995
56005,20040114X00065,Accident,LAX04CA070,2003-12-18,"SAN JOSE, CA",United States,37.332778,-121.819723,RHV,Reid-Hillview,...,Personal,NaN,NaN,NaN,NaN,1.0,VMC,Landing,Probable Cause,30-03-2004


Initial observations:

- 88888 rows and 31 columns
- 5 floats and 26 objects
- Column specifics denoted by a '.' followed by specification ('Airport.Code' & 'Airport.Name')
- Columns that may need to be adjusted to different dtype
    - Accident.Number -> int or float
    - Event.Date -> date type
    - Publication.Date -> date type
- Columns that will need capitalization adjusted
    - Location
    - Air.carrier
    - 

In [5]:
# Assess the na

ad.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          13771
dtype: i

In [6]:
# Assess missingness by percentage

ad.isna().sum()/len(ad)*100

Event.Id                   0.000000
Investigation.Type         0.000000
Accident.Number            0.000000
Event.Date                 0.000000
Location                   0.058500
Country                    0.254250
Latitude                  61.320298
Longitude                 61.330423
Airport.Code              43.469946
Airport.Name              40.611324
Injury.Severity            1.124999
Aircraft.damage            3.593246
Aircraft.Category         63.677170
Registration.Number        1.481623
Make                       0.070875
Model                      0.103500
Amateur.Built              0.114750
Number.of.Engines          6.844491
Engine.Type                7.961615
FAR.Description           63.974170
Schedule                  85.845268
Purpose.of.flight          6.965991
Air.carrier               81.271023
Total.Fatal.Injuries      12.826109
Total.Serious.Injuries    14.073732
Total.Minor.Injuries      13.424608
Total.Uninjured            6.650992
Weather.Condition          5

#### Initial thoughts

- Might be able to create subset of injuries into one column
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
- Drop missing rows from
    - Location
    - Country
    - Make
    - Model
    - Amateur.Built
    - Injury.Severity
- Columns to drop that do not contribute to insights regarding aircrafts 
    - Latitude
    - Longitude
    - Airport.Code & Airport.Name
        - Cound include in Next Steps (which airports to focus on and pull separate data on that)
    - Schedule
    - Purpose.of.flight (is this something that can be helpful for next stage of business development?)
- Create subset analysis
    - Weather.Condition (how do aircrafts wheather in harsh conditions?)

##### Explore columns before deciding how to handle
- Aircraft.Category
- Registration.Number
- Number.of.Engins
- Engin.Type
- FAR.Description
- Schedule
- Air.Carrier
- Broad.phase.of.flight
- Report.Status
- Publication.Date (vs Event.Date which has 0 na)

#### Exploration of Column Values

In [7]:
# Relevant to analysis --> Keep

""" 

An accident is when damage is done to an aircraft or person whereas 
an incident is an event the could have caused an accident.


"""

ad['Investigation.Type'].value_counts()

Accident    85015
Incident     3874
Name: Investigation.Type, dtype: int64

In [8]:
# Not relevant to analysis --> Drop

ad['Schedule'].value_counts()

NSCH    4474
UNK     4099
SCHD    4009
Name: Schedule, dtype: int64

In [9]:
# Lowercase all letters

ad['Make'].value_counts()

Cessna             22227
Piper              12029
CESSNA              4922
Beech               4330
PIPER               2841
                   ...  
Leonard Walters        1
Maule Air Inc.         1
Motley Vans            1
Perlick                1
ROYSE RALPH L          1
Name: Make, Length: 8237, dtype: int64

In [10]:
# Can imputate and na to 'Unknown' & adjust capitalization --> Keep

ad['Purpose.of.flight'].value_counts()

Personal                     49448
Instructional                10601
Unknown                       6802
Aerial Application            4712
Business                      4018
Positioning                   1646
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    405
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: Purpose.of.flight, dtype: int64

In [11]:
# Can imputate and na to 'Unknown' & adjust capitalization --> Keep

ad['Aircraft.damage'].value_counts()

Substantial    64148
Destroyed      18623
Minor           2805
Unknown          119
Name: Aircraft.damage, dtype: int64

In [12]:
# Can imputate and na to 'Unknown' & adjust capitalization --> Keep

ad['Aircraft.Category'].value_counts()

Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

In [13]:
# Not relevant for analysis --> Drop

ad['Registration.Number'].value_counts()

NONE      344
UNREG     126
None       65
UNK        13
USAF        9
         ... 
N93478      1
N519UA      1
N8840W      1
N21040      1
N9026P      1
Name: Registration.Number, Length: 79105, dtype: int64

In [14]:
# Needed for analysis --> Keep
# Missing ~7% of data
# Distribute na values based on probabily distribution of existing values

ad['Number.of.Engines'].value_counts(normalize=True)

1.0    0.840312
2.0    0.133796
0.0    0.014806
3.0    0.005833
4.0    0.005205
8.0    0.000036
6.0    0.000012
Name: Number.of.Engines, dtype: float64

#### Adjust dtype, Capitalization, and Imputate na to Separate Category

In [15]:
# Dropping rows of columns with few missing data
# Will not impact overall analysis

ad.dropna(inplace=True, subset=['Location', 'Country', 'Make', 'Model','Amateur.Built','Injury.Severity'])
ad.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                      0
Country                       0
Latitude                  53338
Longitude                 53347
Airport.Code              37496
Airport.Name              34991
Injury.Severity               0
Aircraft.damage            2639
Aircraft.Category         56103
Registration.Number        1178
Make                          0
Model                         0
Amateur.Built                 0
Number.of.Engines          5212
Engine.Type                6136
FAR.Description           56309
Schedule                  75464
Purpose.of.flight          5132
Air.carrier               71390
Total.Fatal.Injuries      11299
Total.Serious.Injuries    12378
Total.Minor.Injuries      11797
Total.Uninjured            5813
Weather.Condition          3480
Broad.phase.of.flight     25991
Report.Status              5442
Publication.Date          13415
dtype: i

In [16]:
# Adjust na value to it's own category

ad['Purpose.of.flight'].fillna('Unknown', inplace=True)
ad['Aircraft.damage'].fillna('Unknown', inplace=True)
ad['Aircraft.Category'].fillna('Unknown', inplace=True)
print(ad['Purpose.of.flight'].isna().sum())
print(ad['Aircraft.damage'].isna().sum())
print(ad['Aircraft.Category'].isna().sum())

0
0
0


In [17]:
# Adjust capitalization to lowercase

# Create function to lowercase selected columns

def lowercase(df):
    df = df.apply(lambda x: x.str.lower())
    return df

# Imutate on database to lowercase all relevant columns

ad[['Purpose.of.flight','Make','Aircraft.Category']] = lowercase(ad[['Purpose.of.flight','Make','Aircraft.Category']])
ad.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


#### Higher Level Imputation (drop cols, impute on na with stat functions)

###### Create copy of original data for column dropping

TURN THIS BACK TO CODE ONCE DONE WITH OTHER ANALYSIS

"""
These columns do not add value to analysis and are missing alot of data
    - Latitude
    - Longitude
    - Airport.Code
    - Airport.Name
    - Schedule
    
Cound include in Next Steps (which airports to focus on and pull separate data on that)
"""

ad_1 = ad.drop(['Airport.Code', 'Airport.Name', 'Schedule','Restristration.Number'],axis=1)

##### Missing ~7% of data

ad['Number.of.Engines'].value_counts(normalize=True)

Original percenatage:

- 1.0 = 0.843301
- 2.0 = 0.131095
- 0.0 = 0.014839
- 3.0 = 0.005778
- 4.0 = 0.004938
- 8.0 = 0.000036
- 6.0 = 0.000012

After distribution of na:
- 1.0 = 0.843790
- 2.0 = 0.130692
- 0.0 = 0.014778
- 3.0 = 0.005662
- 4.0 = 0.005033
- 8.0 = 0.000034
- 6.0 = 0.000011

In [20]:
# Observation: account for ~7% of the data
# Method: randomly select a class acccording to current distribution

# REMINDER: adjust dataframe name based on the rest of the doc

# Create copy of data base since this is a major imputation
ad_draft = ad.copy()

# relative probabilities (likelihood of 3 dif classes)
rel_prob = [0.843301, 0.131095, 0.014839, 0.005778, 0.004938, 0.000036, 0.000012]

# relative probabilities (likelihood of 3 dif classes)
prob = [i/sum(rel_prob) for i in rel_prob]

# actual probability of each class (x/total)
def impute_pclass(value): # impute = replace
    if pd.isna(value) == True:
        return np.random.choice([1.0,2.0,0.0,3.0,4.0,8.0,6.0], p=prob)
    else:
        return value
        
ad_draft['Number.of.Engines'] = ad_draft['Number.of.Engines'].map(lambda x: impute_pclass(x))
ad_draft['Number.of.Engines'].value_counts(normalize=True)

1.0    0.843790
2.0    0.130692
0.0    0.014778
3.0    0.005662
4.0    0.005033
8.0    0.000034
6.0    0.000011
Name: Number.of.Engines, dtype: float64

In [22]:
ad_draft['Number.of.Engines'].isna().sum()

0